<a href="https://colab.research.google.com/github/GuysBarash/Genetic-programing-with-DEAP/blob/master/evolutionarty_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
colab_mode = 'google.colab' in sys.modules

In [22]:
if colab_mode:
  !apt install libgraphviz-dev
  !pip install pygraphviz
  import pygraphviz as pgv
  !pip install DEAP

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgraphviz-dev is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [0]:
import os
import random
import operator
import numpy as np
import pandas as pd
from datetime import datetime

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from deap.tools import History

from IPython.display import display
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, recall_score, precision_score, fbeta_score

# Generate Data

Fetch data

In [0]:
if not colab_mode:
  train_set = r"C:\school\evolutionary\ex2\train.csv"
  vld_set = r"C:\school\evolutionary\ex2\validate.csv"
  test_set = r"C:\school\evolutionary\ex2\test.csv"
  print("loading trains set.")
  rawdatadf = pd.read_csv(train_set, header=None)
  print("loading VLD set.")
  rawvlddf = pd.read_csv(vld_set, header=None)
  print("loading test set.")
  rawtestdf = pd.read_csv(test_set, header=None)
else:
  from google.colab import drive

  drive.mount('/content/drive')
  base_folder = r'/content/drive/My Drive/colab_storage'
  train_set = os.path.join(base_folder ,'train.csv')
  vld_set = os.path.join(base_folder ,'validate.csv')
  test_set = os.path.join(base_folder ,'test.csv')
  print("Loading train set..")
  rawdatadf = pd.read_csv(train_set, header=None)
  print("Loading validation set.")
  rawvlddf = pd.read_csv(vld_set, header=None)
  print("Loading test set.")
  rawtestdf = pd.read_csv(test_set, header=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading train set..


Choose only last n tuples

In [0]:
last_n_samples = 15

def get_last_n_tuples_from_cols(n, cols):
    return cols[-4 * n:]


datacols = (rawdatadf.columns[1:])
datacols = get_last_n_tuples_from_cols(last_n_samples, datacols)
datadf = pd.DataFrame(columns=datacols, index=range(rawdatadf.shape[0]), data=rawdatadf[datacols])
vld_df = pd.DataFrame(columns=datacols, index=range(rawvlddf.shape[0]), data=rawvlddf[datacols])
test_df = pd.DataFrame(columns=datacols, index=range(rawtestdf.shape[0]), data=rawtestdf[datacols])

Normalize data

In [0]:
ldf = datadf.copy()
mean_of_df = ldf.mean(axis=0)
std_of_df = ldf.std(axis=0)
ldf -= mean_of_df
ldf /= std_of_df
normal = pd.Series(index=ldf.columns, data=np.fmax(-ldf.min(axis=0), ldf.max(axis=0)))
ldf /= normal

def normalize(dataset):
  dataset -= mean_of_df
  dataset /= std_of_df
  # dataset /= normal
  return dataset

datadf = normalize(datadf)
vld_df = normalize(vld_df)
test_df = normalize(test_df)

labels = rawdatadf[rawdatadf.columns[0]]
vld_labels = rawvlddf[rawvlddf.columns[0]]
test_labels = rawtestdf[rawtestdf.columns[0]]
labelcol = 'LABEL'
datadf[labelcol] = labels
vld_df[labelcol] = vld_labels
datadf[labelcol] = labels
test_df[labelcol] = test_labels

Display

In [0]:
top_n = 10
print("Train set.")
display(datadf.head(top_n))
print(f"\nTrain Set: Displaying top {top_n} out of {len(datadf)}.")
print("\n\n")

print("Validation set.")
display(vld_df.head(top_n))
print(f"\nValidation Set: Displaying top {top_n} out of {len(vld_df)}.")
print("\n\n")

print("test set.")
display(test_df.head(top_n))
print(f"\Test Set: Displaying top {top_n} out of {len(test_df)}.")
print("\n\n")

Data folding. 


*   Sample: an entry from the data
*   event: number of items in each sample (15 as defualt) 
*   feature: number of measuring at each event (4 at defualt) 

After folding:                                           
X = (#samples x events x features).                
y = [#is_label_0, is_label_1]

In [0]:
def fold_data(dataset, features=4):
    entries_per_sample = int(len(datacols) / features)
    samples = dataset.shape[0]
    X = np.zeros((samples, entries_per_sample, features))
    for pos in range(entries_per_sample):
        entry = dataset[datacols[features * pos:features * pos + features]]
        X[:, pos, :] = entry
    y = pd.get_dummies(dataset[labelcol])
    return X, y

number_of_features = 4
window_size = int(len(datacols) / number_of_features)
X , y =  fold_data(datadf, number_of_features)

print(f"X: {X.shape}")
print(f"y: {y.shape}")
print(f"Example of a single sample:")
display(pd.DataFrame(data=X[0,:,:]))

In [0]:
X_train, y_train  = fold_data(datadf.iloc[:100000], number_of_features)
X_vld , y_vld  = fold_data(vld_df, number_of_features)
X_test , y_test  = fold_data(test_df, number_of_features)
# X_train, X_vld, y_train, y_vld = train_test_split(X, y, test_size=0.2)
# X_vld, X_test, y_vld, y_test = train_test_split(X_vld, y_vld, test_size=0.01)

print(f"Train\t: {X_train.shape}")
print(f"VLD\t: {X_vld.shape}")
print(f"Test\t: {X_test.shape}")
print(f"original X: {X.shape}")

In [0]:
X_source = X_train
y_source = y_train

labels_1_idx = y_source[y_source[1] == 1].index
labels_0_idx = y_source[y_source[1] == 0].index

print(f"Positives: {labels_1_idx.shape[0]}")
print(f"Negatives: {labels_0_idx.shape[0]}")

all_positive = X_source[labels_1_idx,:,:]
all_negative = X_source[labels_0_idx,:,:]
all_positive_mean = all_positive.mean(axis=0)
all_negative_mean = all_negative.mean(axis=0)


positive_df = pd.DataFrame(index=range(all_positive_mean.shape[0]),data=all_positive_mean)
print("POSITIVES")
display(positive_df)

print("")
print("")
negative_df = pd.DataFrame(index=range(all_negative_mean.shape[0]),data=all_negative_mean)
print("POSITIVES")
display(negative_df)

In [0]:
  fig_limits = (-0.2,+0.2)
  plt.figure(figsize=(20, 10))
  plt.figure(1)
  plt.subplot(411)
  plt.plot(positive_df[0], color='red', label='positive')
  plt.plot(negative_df[0], color='black', label='negative')
  plt.ylim(fig_limits[0], fig_limits[1])
  plt.subplot(412)
  plt.plot(positive_df[1], color='red', label='positive')
  plt.plot(negative_df[1], color='black', label='negative')
  plt.ylim(fig_limits[0], fig_limits[1])
  plt.subplot(413)
  plt.plot(positive_df[2], color='red', label='positive')
  plt.plot(negative_df[2], color='black', label='negative')
  plt.ylim(fig_limits[0], fig_limits[1])
  plt.subplot(414)
  plt.plot(positive_df[3], color='red', label='positive')
  plt.plot(negative_df[3], color='black', label='negative')
  plt.ylim(fig_limits[0], fig_limits[1])
  plt.legend()
  _ = plt.plot()

  for idx in range(4):
    print(f"[POS][Col {idx}]: {positive_df[idx].mean():.3f}+{positive_df[idx].std():.3f}")
    print(f"[NEG][Col {idx}]: {negative_df[idx].mean():.3f}+{negative_df[idx].std():.3f}")
    print("")

# Defining evolution parameters

register operands

In [0]:
def activation(a):
    return np.tanh(a)


def neg(a):
    return -a


def double(a):
    return 2.0 * a


def half(a):
    return 0.5 * a

In [0]:
pset = gp.PrimitiveSet("MAIN", 4, "IN")
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(np.fmax, 2)
pset.addPrimitive(np.square, 1)
pset.addPrimitive(neg, 1)
pset.addPrimitive(double, 1)
pset.addPrimitive(half, 1)
pset.addPrimitive(activation, 1)
pset.addTerminal(np.float64(1.0))
pset.addTerminal(np.float64(0.25))
pset.addTerminal(np.float64(0.5))
pset.addTerminal(np.float64(2.0))
pset.addTerminal(np.float64(3.0))

Support functions

In [0]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [0]:
def individual_to_function(individual):
    return toolbox.compile(expr=individual)

def train_creature(X, y, func):
    model_X = func(X[:, :, 0], X[:, :, 1], X[:, :, 2], X[:, :, 3])
    if np.isscalar(model_X):
        model_X = np.full(X.shape[:2], model_X)
    clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3), verbose=0)
    _ = clf.fit(model_X, y)
    return clf


def predict_creatute(X, clf, func):
    model_X = func(X[:, :, 0], X[:, :, 1], X[:, :, 2], X[:, :, 3])
    if np.isscalar(model_X):
        model_X = np.full(X.shape[:2], model_X)
    y_pred = clf.predict(model_X)
    return y_pred


def train_and_predict(X_train, y_train, X_predict, func):
    clf = train_creature(X_train, y_train, func)
    y_pred = predict_creatute(X_predict, clf, func)
    return y_pred


def fitness_function(individual):
    global evaluation_counter
    evaluation_counter += 1

    dynamic_train_idx = np.random.choice(X_train.shape[0], size=10000)
    # dynamic_vld_idx = np.random.choice(X_train.shape[0], size=1000)
    dynamic_X_train = X_train[dynamic_train_idx, :, :]
    dynamic_y_train = y_train.loc[dynamic_train_idx, 1]

    func = individual_to_function(individual)
    y_pred = train_and_predict(dynamic_X_train, dynamic_y_train, dynamic_X_train, func)
    if np.unique(y_pred).shape[0] < 2:
        # All elemetns are identical
        return 0.0,
    else:
        # Fitness is accuracy
        # fitness_score = accuracy_score(dataset_y, results)
        # Fitness is F0.25 score
        fitness_score = fbeta_score(dynamic_y_train, y_pred, beta=0.25)
        return fitness_score,

In [0]:
calculations_time = datetime.now()


def calc_time(*args):
    global calculations_time
    now = datetime.now()
    result_time = now - calculations_time
    calculations_time = now
    return result_time

Register hyper parameters

In [0]:
# random.seed(10)
population_size = 50
number_of_generations = 500

In [0]:
history = History()

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=2, max_=8)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", fitness_function)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genGrow, min_=0, max_=8)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# Bloat control
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=10))

pop = toolbox.population(n=population_size)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
stats.register("TIME", calc_time)

# Decorate the variation operators
toolbox.decorate("mate", history.decorator)
toolbox.decorate("mutate", history.decorator)
history.update(pop)

# Evolve

In [0]:
print("Starting evolution.")
evaluation_counter = 0
start_time = datetime.now()
calculations_time = datetime.now()
_, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.15, ngen=number_of_generations, stats=stats, halloffame=hof)

# Results

In [0]:
record = stats.compile(pop)
duration = datetime.now() - start_time
print("Run concluded.")
print("Evaluations commited: {}".format(evaluation_counter))
print("Run time: {}".format(duration))
winner_creature = hof.items[0]
winner_function = individual_to_function(winner_creature)

In [0]:
nodes, edges, labels = gp.graph(winner_creature)

In [0]:
if colab_mode:
  import matplotlib.pyplot as plt
  import networkx as nx
  from networkx.drawing.nx_agraph import graphviz_layout

  plt.rcParams["figure.figsize"] = (50, 40)

  g = nx.Graph()
  g.add_nodes_from(nodes)
  g.add_edges_from(edges)
  pos = graphviz_layout(g, prog="dot")


  nx.draw_networkx_nodes(g, pos, node_size=1600)
  nx.draw_networkx_edges(g, pos)
  nx.draw_networkx_labels(g, pos, labels, font_size=25)
  plt.show()

analyze and display

In [0]:
generations_idx = logbook.select('gen')
generations_avg = logbook.select('avg')
generations_std = logbook.select('std')
generations_max = logbook.select('max')
generations_min = logbook.select('min')

plt.figure(figsize=(20, 10))
plt.errorbar(generations_idx, generations_avg, yerr=generations_std, fmt='-o', label='AVG')
# plt.plot(generations_idx,generations_avg, '-o',label='AVG')
plt.plot(generations_max, '-o', label='BEST', color='black')
plt.plot(generations_min, '-o', label='WORST', color='red')
plt.grid()
plt.legend()
plt.xlabel('Generations')
plt.ylabel('hits')
plt.ylim(0, 1)
if colab_mode:
  _ = plt.plot()
else:
  plt.savefig("Evolution.png")

Evaluate winner on entire train-set

In [0]:
predictions = train_and_predict(X_train, y_train[1], X_train, winner_function)
true_results = y_train[1]

scoring_sr = pd.DataFrame(dtype=np.float, columns=['value'])
scoring_sr.loc['Accuracy', 'value'] = accuracy_score(true_results, predictions)
scoring_sr.loc['Recall', 'value'] = recall_score(true_results, predictions)
scoring_sr.loc['precision', 'value'] = precision_score(true_results, predictions)
scoring_sr.loc['F0.25', 'value'] = fbeta_score(true_results, predictions, beta=0.25)

print("Labels balance:")
print("0: {:>.3f}".format(1 - predictions.mean()))
print("1: {:>.3f}".format(predictions.mean()))
display(scoring_sr)

Evaluate winner on VLD group

In [0]:
predictions = train_and_predict(X_train, y_train[1], X_vld, winner_function)
true_results = y_vld[1]

scoring_sr = pd.DataFrame(dtype=np.float, columns=['value'])
scoring_sr.loc['Accuracy', 'value'] = accuracy_score(true_results, predictions)
scoring_sr.loc['Recall', 'value'] = recall_score(true_results, predictions)
scoring_sr.loc['precision', 'value'] = precision_score(true_results, predictions)
scoring_sr.loc['F0.25', 'value'] = fbeta_score(true_results, predictions, beta=0.25)

print("Labels balance:")
print("0: {:>.3f}".format(1 - predictions.mean()))
print("1: {:>.3f}".format(predictions.mean()))

display(scoring_sr)